In [ ]:
# What do we do about the SIC codes?
# Search for "imbalanced class" for imbalanced class setup

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
import gc

In [2]:
data = pd.read_csv('stock_price_data.csv')
first_day = pd.to_datetime('2024-1-3')
last_day = pd.to_datetime('2024-2-13')
num_of_ts = 29  # Number of timestamps; later remove one because the last timestamp is for testing
seq_length = 20 # Length of time-series
num_of_classes = 10
# 5 classes for imbalanced class setup
# num_of_classes = 5 # Top 2 and bottom 2 are two and bottom 2%; middle class is remaining 96%
seed = 120 # To set global seed later for reproducible results

# Some stocks are double listed, so company name cannot differentiate among them
# Tickers seem to be enough; number of unique tickers = 6345

# number of unique company names = 6293

# data['id'] = data['GVKEY'] + 0.1*data['iid']
# number of unique data['id'] = 6344; WHY IS IT LESS??

tickers = data['tic'].unique()
print(f'There are {len(tickers)} unique tickers in raw data')

# For grouping later
data['datadate'] = pd.to_datetime(data['datadate'], format='%m/%d/%Y')

# Remove entries where the dates are in the future
data = data[data['datadate'] < last_day]

print(f'Raw data has shape {data.shape}')

There are 6345 unique tickers in raw data
Raw data has shape (181872, 13)


In [3]:
cols = list(data.columns)
factors = list(set(data.columns).difference({'GVKEY', 'iid', 'datadate', 'tic', 'conm', 'sic'}))
print(f'all columns {cols}')
print(f'all factors {factors}')

print(f'columns with missing data are {data.columns[data.isnull().sum() != 0]}')

all columns ['GVKEY', 'iid', 'datadate', 'tic', 'conm', 'cshoc', 'cshtrd', 'eps', 'prccd', 'prchd', 'prcld', 'prcod', 'sic']
all factors ['eps', 'prcld', 'prchd', 'cshtrd', 'prccd', 'cshoc', 'prcod']
columns with missing data are Index(['cshoc', 'cshtrd', 'eps', 'prccd', 'prchd', 'prcld', 'prcod'], dtype='object')


In [4]:
def remove_tic(data, tickers):
    # Remove entries without key factors such as trading volume, open/close prices
    to_remove = []
    for f in factors:
        factor_missing = data[data[f].isna()] # rows with factor f missing
        tic_missing = list(factor_missing['tic'].unique()) # find all tickers with missing f
        to_remove = to_remove + tic_missing # add tickers to the list of tickers that will be removed
    print(f'There are {len(set(to_remove))} tickers to remove due to missing factors')
    data = data[~data['tic'].isin(to_remove)]
    
    # Remove entries where there are not enough number of time stamps
    databytic_count = data.groupby('tic').count()
    incomplete_tics = databytic_count.loc[databytic_count['datadate']<num_of_ts].index.tolist()
    data = data[~data['tic'].isin(incomplete_tics)]
    tickers = data['tic'].unique()
    print(f'There are {len(incomplete_tics)} unique tickers to remove due to insufficient timestamps')
    print('There are ' + str(len(tickers)) + ' tickers')

    print(f'Confirm there are no more columns with missing data {data.columns[data.isnull().sum() != 0]}')
    
    # Sort data by tickers and date
    data = data.sort_values(by=['tic','datadate'])
    data = data.reset_index(drop=True)
    
    # Miscellaneous
    tic_comp_dict = dict(zip(data.tic, data.conm))
    data.insert(3, 'tic_num', pd.factorize(data['tic'])[0])
    
    return data, tickers, tic_comp_dict

data, tickers, tic_comp_dict = remove_tic(data, tickers)

There are 1822 tickers to remove due to missing factors
There are 31 unique tickers to remove due to insufficient timestamps
There are 4492 tickers
Confirm there are no more columns with missing data Index([], dtype='object')


In [5]:
# Compute a new column: percentage daily return (ret_d)
def compute_ret(data):
    # Add additional column 'ret_d'
    data['ret_d']=data.groupby('tic')['prccd'].pct_change()
    # Shift all numbers up by one day. e.g. 2024-2-2's ret_d is the percentage return of 2024-2-3's daily return
    # It is easier for training this way since we want to predict next day's return
    data['ret_d'] = data.groupby('tic')['ret_d'].shift(-1)

    # Currently the last day has no ret_d since it's the test date. Fill it with 0 (optional)
    data = data.fillna(0)
    data = data.reset_index(drop=True)
    
    return data

data = compute_ret(data)

print(f'Final data has shape {data.shape}')

# Make sure every ticker has exactly num_of_ts timestamps
assert len(tickers) * num_of_ts == data.shape[0]

Final data has shape (130268, 15)


In [6]:
# Give each ticker a numerical label for easier reference, stored in a dictionary
all_tickers = list(data['tic'].unique())
all_tickers.sort()
def num_to_tic(all_tickers):
    l = len(all_tickers)
    num_to_tic_dict = {}
    for i in range(l):
        num_to_tic_dict[i] = all_tickers[i]
    return num_to_tic_dict

num_to_tic_dict = num_to_tic(all_tickers)

In [7]:
# Optionally look at the first two companies
data.head(2*num_of_ts)

,GVKEY,iid,datadate,tic_num,tic,conm,cshoc,cshtrd,eps,prccd,prchd,prcld,prcod,sic,ret_d
0,126554,1,2024-01-02,0,A,AGILENT TECHNOLOGIES INC,293004000.0,1441580.0,4.22,138.75,140.5900,137.9100,138.19,3826,-0.054703
1,126554,1,2024-01-03,0,A,AGILENT TECHNOLOGIES INC,293004000.0,2074454.0,4.22,131.16,138.0000,131.0650,138.00,3826,-0.001220
2,126554,1,2024-01-04,0,A,AGILENT TECHNOLOGIES INC,293004000.0,2446585.0,4.22,131.00,131.4950,130.1900,130.55,3826,-0.003359
3,126554,1,2024-01-05,0,A,AGILENT TECHNOLOGIES INC,293004000.0,1393963.0,4.22,130.56,131.9600,128.6200,130.00,3826,0.021599
4,126554,1,2024-01-08,0,A,AGILENT TECHNOLOGIES INC,293004000.0,1311358.0,4.22,133.38,133.5700,129.8100,130.14,3826,-0.020243
5,126554,1,2024-01-09,0,A,AGILENT TECHNOLOGIES INC,293004000.0,1434952.0,4.22,130.68,135.6450,130.0100,132.27,3826,0.003137
6,126554,1,2024-01-10,0,A,AGILENT TECHNOLOGIES INC,293004000.0,1326296.0,4.22,131.09,131.1600,128.3600,130.58,3826,-0.010756
7,126554,1,2024-01-11,0,A,AGILENT TECHNOLOGIES INC,293004000.0,2060521.0,4.22,129.68,130.6800,127.9000,130.58,3826,0.006632
8,126554,1,2024-01-12,0,A,AGILENT TECHNOLOGIES INC,293004000.0,1285203.0,4.22,130.54,131.6100,129.6400,130.31,3826,-0.000077
9,126554,1,2024-01-16,0,A,AGILENT TECHNOLOGIES INC,293004000.0,1382120.0,4.22,130.53,130.8100,128.5950,129.14,3826,-0.018463


In [8]:
# All traded days in this dataset
all_days = list(data['datadate'].unique())
nt = len(tickers)

# Organize y_train as categorical data and use cross entropy instead of sparse cross entropy
def prepare_data(data, seq_length):
    # Block 1: standardize all data cross-sectionally
    # data_sd = data[factors + ['datadate']].groupby('datadate').transform(lambda x: (x - x.mean()) / x.std())
    # data[factors] = data_sd
    # Block 2: cross sectional and per ticker
    # Cross-sectional standardization
    data_cs = data[['cshoc', 'cshtrd', 'eps', 'datadate']].groupby('datadate').transform(lambda x: (x - x.mean()) / x.std())
    # Tic-level standardization
    data_ts = data[['prccd', 'prchd', 'prcld', 'prcod', 'tic']].groupby('tic').transform(lambda x: (x - x.mean()) / x.std())
    data[['cshoc', 'cshtrd', 'eps']] = data_cs
    data[['prccd', 'prchd', 'prcld', 'prcod']] = data_ts
    print(f'Confirm that standardization did not create NaNs: {data.columns[data.isnull().sum() != 0]}')
    
    # Separate training data to construct the rank column
    data_train = data[data['datadate'] != pd.to_datetime(all_days[-1])].reset_index(drop=True)
    
    # Compute how many training data we will have
    # Testing data are always the last day
    num_train_data = (num_of_ts - seq_length) * nt
    num_test_data = nt
    
    # Create ret_d_rank column as labels
    data_train['ret_d_DistinctRank'] = data_train.groupby('datadate')['ret_d'].rank(method='first')
    data_train['ret_d_rank'] = data_train.groupby('datadate')['ret_d_DistinctRank'].transform(
                                                                            lambda x: pd.qcut(x, 10, labels=range(10)))
    # Uncomment the next two lines for imbalanced class setup
    #data_train['ret_d_rank'] = data_train['ret_d_rank'].replace(range(2, 8), 2)
    #data_train['ret_d_rank'] = data_train['ret_d_rank'].replace({8: 3, 9: 4})
    
    # Create training data
    x_train = np.zeros((num_train_data, len(factors), seq_length))
    y_train = np.zeros((num_train_data, ))
    for i in range(num_of_ts - seq_length):
        train_days = all_days[i : seq_length + i]
        data_temp = data_train[data_train['datadate'].isin(train_days)]
        # Convert dataframe data to three dimensional training data (ticker, factor, time-series data)
        # This is 'channels_first' type of data in training!; will change to channel last later!
        pivot_data = data_temp[factors+['datadate', 'tic']].pivot_table(index='tic', columns='datadate')
        x_train[i*nt:(i+1)*nt, :, :] = pivot_data.values.reshape(nt, len(factors), seq_length)
        y_train[i*nt:(i+1)*nt] = data_train[data_train['datadate'] == train_days[-1]]['ret_d_rank'].values.reshape(nt, )
    
    # Create testing data
    test_days = all_days[num_of_ts-seq_length:]
    data_temp = data[data['datadate'].isin(test_days)]
    pivot_data = data_temp[factors+['datadate', 'tic']].pivot_table(index='tic', columns='datadate')
    x_test = pivot_data.values.reshape(num_test_data, len(factors), seq_length)
    
    # Reshape train/test data so that it is channels_last
    x_train = np.transpose(x_train, (0, 2, 1))
    x_test = np.transpose(x_test, (0, 2, 1))
    
    assert x_train.shape[0] == num_train_data
    assert x_test.shape[0] == num_test_data
    
    print(f'Training data have shape {x_train.shape}, {y_train.shape}')
    print(f'Testing data have shape {x_test.shape}')
    
    # Can consider returning data_train; data has no rank column
    return data, data_train, x_train, y_train, x_test # Returned data is standardized

data, data_train, x_train, y_train, x_test = prepare_data(data, seq_length)

Confirm that standardization did not create NaNs: Index([], dtype='object')
Training data have shape (40428, 20, 7), (40428,)
Testing data have shape (4492, 20, 7)


In [ ]:
y_train

# Train CNN

In [16]:
dropout_rate = 0.2

def make_model(input_shape):
    input_layer = keras.layers.Input(input_shape)
    
    # Block 1: easy architecture
#     x = keras.layers.Conv1D(filters=8, kernel_size=3, padding="same")(input_layer)
#     x = keras.layers.BatchNormalization()(x)
#     x = keras.layers.ReLU()(x)
#     x = keras.layers.Dropout(dropout_rate)(x)
#     x = keras.layers.Conv1D(filters=32, kernel_size=3, padding="same")(x)
#     x = keras.layers.BatchNormalization()(x)
#     x = keras.layers.ReLU()(x)
#     x = keras.layers.Dropout(dropout_rate)(x)
#     x = keras.layers.GlobalAveragePooling1D()(x)
    
    # Block 2: medium architecture
    x = keras.layers.Conv1D(filters=32, kernel_size=5, padding="same")(input_layer)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.ReLU()(x)
    x = keras.layers.Dropout(dropout_rate)(x)

    x = keras.layers.Conv1D(filters=64, kernel_size=5, padding="same")(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.ReLU()(x)
    x = keras.layers.Dropout(dropout_rate)(x)

    x = keras.layers.Conv1D(filters=128, kernel_size=5, padding="same")(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.ReLU()(x)
    x = keras.layers.Dropout(dropout_rate)(x)
    x = keras.layers.GlobalAveragePooling1D()(x)
    
    # Uses softmax here because the output is a distribution of the two classes
    x = keras.layers.Dense(64, activation='relu')(x)
    x = keras.layers.Dropout(dropout_rate)(x)
    x = keras.layers.Dense(16, activation='relu')(x)
    x = keras.layers.Dropout(dropout_rate)(x)
    output_layer = keras.layers.Dense(num_of_classes, activation="softmax")(x)

    return keras.models.Model(inputs=input_layer, outputs=output_layer)


model = make_model(input_shape=x_train.shape[1:])

In [17]:
epochs = 500
batch_size = 128
lr_val = 1e-3

# TRAINS FOR LONG DUE TO DOUBLE DESCENT?
callbacks = [
    # Saves the best model according to val_loss; check at every iteration
    keras.callbacks.ModelCheckpoint(
        "best_model.keras", save_best_only=True, monitor="val_loss"
    ),
    # Reduce the learning rate by the factor if val_loss does not improve after 20 iterations
    keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss", factor=0.5, patience=30, min_lr=1e-5
    ),
    # Stops training if validation does not improve after 50 iterations
    keras.callbacks.EarlyStopping(monitor="val_loss", patience=100, verbose=1),
]

# Uncomment the following for imbalanced class setup
# class_weights = {0: 100, 1: 100, 2: 1, 3: 100, 4: 100}
# unweighted_loss = tf.keras.losses.SparseCategoricalCrossentropy()
# Define customized sparse cross entropy with class weights
# def weighted_loss(y_true, y_pred):
#     loss = unweighted_loss(y_true, y_pred)
#     class_weights_tensor = tf.constant(list(class_weights.values()), dtype=tf.float32)
#     weights = tf.gather(class_weights_tensor, tf.cast(y_true, dtype=tf.int32))
#     weighted_loss = loss * weights
#     return weighted_loss

model.compile(
    optimizer=keras.optimizers.Adam(lr_val),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    # imbalanced class setup
    #loss=weighted_loss,
    metrics=["sparse_categorical_accuracy"]
)

model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 20, 7)]           0         
                                                                 
 conv1d_3 (Conv1D)           (None, 20, 32)            1152      
                                                                 
 batch_normalization_3 (Batc  (None, 20, 32)           128       
 hNormalization)                                                 
                                                                 
 re_lu_3 (ReLU)              (None, 20, 32)            0         
                                                                 
 dropout_5 (Dropout)         (None, 20, 32)            0         
                                                                 
 conv1d_4 (Conv1D)           (None, 20, 64)            10304     
                                                           

In [ ]:
history = model.fit(
    x_train,
    y_train,
    batch_size=batch_size,
    epochs=epochs,
    callbacks=callbacks,
    validation_split=0.2,
    verbose=1,
)

Epoch 1/500
253/253 [==============================] - 7s 22ms/step - loss: 2.2882 - sparse_categorical_accuracy: 0.1224 - val_loss: 2.2895 - val_sparse_categorical_accuracy: 0.1386 - lr: 0.0010
Epoch 2/500
253/253 [==============================] - 6s 22ms/step - loss: 2.2561 - sparse_categorical_accuracy: 0.1509 - val_loss: 2.2565 - val_sparse_categorical_accuracy: 0.1456 - lr: 0.0010
Epoch 3/500
253/253 [==============================] - 5s 21ms/step - loss: 2.2194 - sparse_categorical_accuracy: 0.1697 - val_loss: 2.2456 - val_sparse_categorical_accuracy: 0.1592 - lr: 0.0010
Epoch 4/500
253/253 [==============================] - 5s 21ms/step - loss: 2.1931 - sparse_categorical_accuracy: 0.1784 - val_loss: 2.2444 - val_sparse_categorical_accuracy: 0.1506 - lr: 0.0010
Epoch 5/500
253/253 [==============================] - 6s 24ms/step - loss: 2.1794 - sparse_categorical_accuracy: 0.1820 - val_loss: 2.2500 - val_sparse_categorical_accuracy: 0.1572 - lr: 0.0010
Epoch 6/500
253/253 [====

Epoch 43/500
253/253 [==============================] - 6s 22ms/step - loss: 2.0746 - sparse_categorical_accuracy: 0.2245 - val_loss: 2.2106 - val_sparse_categorical_accuracy: 0.1757 - lr: 0.0010
Epoch 44/500
253/253 [==============================] - 6s 22ms/step - loss: 2.0706 - sparse_categorical_accuracy: 0.2275 - val_loss: 2.2017 - val_sparse_categorical_accuracy: 0.1815 - lr: 0.0010
Epoch 45/500
253/253 [==============================] - 6s 25ms/step - loss: 2.0672 - sparse_categorical_accuracy: 0.2278 - val_loss: 2.1983 - val_sparse_categorical_accuracy: 0.1832 - lr: 0.0010
Epoch 46/500
253/253 [==============================] - 8s 33ms/step - loss: 2.0719 - sparse_categorical_accuracy: 0.2275 - val_loss: 2.2128 - val_sparse_categorical_accuracy: 0.1806 - lr: 0.0010
Epoch 47/500
253/253 [==============================] - 7s 26ms/step - loss: 2.0692 - sparse_categorical_accuracy: 0.2275 - val_loss: 2.2194 - val_sparse_categorical_accuracy: 0.1775 - lr: 0.0010
Epoch 48/500
253/253

 37/253 [===>..........................] - ETA: 4s - loss: 2.0209 - sparse_categorical_accuracy: 0.2348

In [12]:
model.evaluate(x_train, y_train)

1264/1264 [==============================] - 3s 3ms/step - loss: 1.7696 - sparse_categorical_accuracy: 0.4014


[1.7696388959884644, 0.401355504989624]

In [13]:
y_test = model.predict(x_test)

141/141 [==============================] - 1s 3ms/step


In [14]:
y_test

array([[1.30739296e-03, 1.55903816e-01, 2.69877225e-01, ...,
        4.26670816e-03, 4.55418522e-05, 7.30647498e-09],
       [5.99381565e-05, 3.17508588e-04, 5.22353628e-04, ...,
        8.01815018e-02, 1.60056069e-01, 7.15331435e-01],
       [3.85021508e-01, 5.55361882e-02, 1.89883802e-02, ...,
        2.27699485e-02, 5.02580442e-02, 4.21559781e-01],
       ...,
       [7.36839890e-01, 2.38431245e-01, 1.64512992e-02, ...,
        5.23362178e-06, 1.30959745e-06, 1.15099601e-08],
       [1.47333443e-01, 3.06170225e-01, 1.74601957e-01, ...,
        1.93419550e-02, 1.33062778e-02, 5.35129337e-04],
       [9.63653684e-01, 3.08607519e-02, 1.79932662e-03, ...,
        1.08039116e-04, 1.27188207e-04, 2.73455244e-05]], dtype=float32)

In [15]:
for i in range(nt):
    if np.argmax(y_test[i, :]) == 0:
        print(f'{num_to_tic_dict[i]} is top stock')

for i in range(nt):
    if np.argmax(y_test[i, :]) == 9:
        print(f'{num_to_tic_dict[i]} is bottom stock')
        
# For imbalanced class setup
# for i in range(nt):
#     if np.argmax(y_test[i, :]) != 2:
#         print(f'{num_to_tic_dict[i]} is rank {np.argmax(y_test[i, :])} stock')

AA is bottom stock
AADI is bottom stock
AAMC is bottom stock
AAN is bottom stock
AAON is top stock
AAP is top stock
AAU is bottom stock
ABAT is bottom stock
ABEO is top stock
ABG is top stock
ABL is top stock
ABLV is bottom stock
ABNB is top stock
ABOS is bottom stock
ABR is bottom stock
ABSI is top stock
ABUS is top stock
ACDC is bottom stock
ACEL is top stock
ACGL is top stock
ACI is bottom stock
ACIC is top stock
ACLX is top stock
ACMR is top stock
ACNB is bottom stock
ACON is top stock
ACRE is bottom stock
ACRS is top stock
ACRV is bottom stock
ACST is bottom stock
ACTG is top stock
ACXP is bottom stock
ADAP is top stock
ADCT is top stock
ADD is bottom stock
ADIL is bottom stock
ADM is bottom stock
ADN is bottom stock
ADP is top stock
ADPT is bottom stock
ADTH is top stock
ADTX is bottom stock
ADUS is top stock
ADVM is top stock
AEHR is bottom stock
AEI is top stock
AEMD is bottom stock
AENT is top stock
AEO is top stock
AEON is top stock
AES is bottom stock
AEVA is top stock
AEY i

In [ ]:
gc.collect()